In [1]:
# copy each chunk of mediapipe_pose.py script from home video analysis folder 
# print outputs so I can see what is happening/where errors might be 

In [1]:
import cv2
import mediapipe as mp
import sys
import pandas as pd
import os

poseDict = {
    0: "nose",
    1: "left_eye_inner",
    2: "left_eye",
    3: "left_eye_outer",
    4: "right_eye_inner",
    5: "right_eye",
    6: "right_eye_outer",
    7: "left_ear",
    8: "right_ear",
    9: "mouth_left",
    10: "mouth_right",
    11: "left_shoulder",
    12: "right_shoulder",
    13: "left_elbow",
    14: "right_elbow",
    15: "left_wrist",
    16: "right_wrist",
    17: "left_pinky",
    18: "right_pinky",
    19: "left_index",
    20: "right_index",
    21: "left_thumb",
    22: "right_thumb",
    23: "left_hip",
    24: "right_hip",
    25: "left_knee",
    26: "right_knee",
    27: "left_ankle",
    28: "right_ankle",
    29: "left_heel",
    30: "right_heel",
    31: "left_foot_index",
    32: "right_foot_index"
}

In [2]:
print(poseDict)

{0: 'nose', 1: 'left_eye_inner', 2: 'left_eye', 3: 'left_eye_outer', 4: 'right_eye_inner', 5: 'right_eye', 6: 'right_eye_outer', 7: 'left_ear', 8: 'right_ear', 9: 'mouth_left', 10: 'mouth_right', 11: 'left_shoulder', 12: 'right_shoulder', 13: 'left_elbow', 14: 'right_elbow', 15: 'left_wrist', 16: 'right_wrist', 17: 'left_pinky', 18: 'right_pinky', 19: 'left_index', 20: 'right_index', 21: 'left_thumb', 22: 'right_thumb', 23: 'left_hip', 24: 'right_hip', 25: 'left_knee', 26: 'right_knee', 27: 'left_ankle', 28: 'right_ankle', 29: 'left_heel', 30: 'right_heel', 31: 'left_foot_index', 32: 'right_foot_index'}


In [3]:
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
mp_pose = mp.solutions.pose

In [4]:
# create inFolderPath and outFolderPath variables set in command prompt 

# full windows path - this works 
# inFolderPath = r'C:\Users\mmccu\Box\MM_Personal\5_Projects\BoveLab\3_Data_and_Code\hva_code\HomeVideoAnalysis\tests\fixtures\videos'
# outFolderPath = r'C:\Users\mmccu\Box\MM_Personal\5_Projects\BoveLab\3_Data_and_Code\hva_code\HomeVideoAnalysis\temp' 

## New: trying relative paths 
# get curent working directory 
current_directory = os.getcwd()
print(current_directory)

# move back to homeVideoAnalysis
inFolderPath = r'..\tests\fixtures\videos'
print(inFolderPath)
outFolderPath = r'..\temp'
print(outFolderPath)

C:\Users\mmccu\Box\MM_Personal\5_Projects\BoveLab\3_Data_and_Code\hva_code\HomeVideoAnalysis\megan_sandbox
..\tests\fixtures\videos
..\temp


In [5]:
#def process_folder(inFolderPath, outFolderPath):
for (dirpath, dirnames, filenames) in os.walk(inFolderPath):
    for filename in filenames:
        name, ext = os.path.splitext(filename)
        ext = ext.lower()[1:]
        if (ext == "mov" or ext == "mp4"):
                inPath = os.path.join(dirpath, filename)
                inPath = os.path.normpath(inPath) ## Megan added - trying to fix issue with windows file path
                outPath = os.path.join(outFolderPath, name)
                outPath = os.path.normpath(outPath) ##Megan added - trying to fix issue with windows file path
                print(f"Processing: {inPath}")
                #process_video(inPath, outPath)

Processing: ..\tests\fixtures\videos\gait_vertical_left.mov


In [6]:
print(inFolderPath)
print(outFolderPath)
print(inPath)
print(outPath)

..\tests\fixtures\videos
..\temp
..\tests\fixtures\videos\gait_vertical_left.mov
..\temp\gait_vertical_left


In [7]:
def renameCols(col):
    lmNum = col.split('_')[0]
    lmVal = poseDict[int(lmNum)]
    return col.replace(lmNum, lmVal)

In [8]:
# def process_video(inFile, outFile, exportVid=True) 

# from proccess_folder() function
inFile = inPath 
outFile = outPath
exportVid = True

#megan edit name - windows 
# name = inFile.split('/')[-1].split('.')[0] - original 
inFile_split = os.path.split(inFile)
inFile_basename = inFile_split[1]
inFile_last_folder = os.path.split(inFile_split[0])
name = os.path.join(inFile_last_folder[1], inFile_basename)
    
data = pd.DataFrame()

In [9]:
print(name)
print(data)

videos\gait_vertical_left.mov
Empty DataFrame
Columns: []
Index: []


In [10]:
print(inFile)
cap = cv2.VideoCapture(inFile)

..\tests\fixtures\videos\gait_vertical_left.mov


In [11]:
print(os.path.normpath(f'./{outFile}_mpVid.mp4'))

..\temp\gait_vertical_left_mpVid.mp4


In [12]:
if exportVid:
    fourcc = cv2.VideoWriter_fourcc('F', 'M', 'P', '4')
    out = cv2.VideoWriter(os.path.normpath(f'./{outFile}_mpVid.mp4'), fourcc, 20, # edited for windows 
                            (int(cap.get(3)),
                             int(cap.get(4))))

In [ ]:
with mp_pose.Pose(
        min_detection_confidence=0.5,
        min_tracking_confidence=0.5) as pose:
     while cap.isOpened():
        success, image = cap.read()
        if not success:
            #print("Ignoring empty camera frame.")
            # If loading a video, use 'break' instead of 'continue'.
            break

        # To improve performance, optionally mark the image as not writeable to
        # pass by reference.
        image.flags.writeable = False
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        results = pose.process(image)
        if results.pose_landmarks:
            tLndMrks = list(map(lambda lndMrk: (lndMrk.x, lndMrk.y,
                                                lndMrk.z, lndMrk.visibility,
                                                lndMrk.presence) if lndMrk else None,
                                results.pose_landmarks.landmark))
        else:
            tLndMrks = None
            tData = None

        if tLndMrks and len(tLndMrks) == 33:
            tData = pd.DataFrame(tLndMrks, columns=['x', 'y', 'z', 'vis', 'pres']).reset_index().stack()
            tData.index = list(map(lambda idx: f"{idx[0]}_{idx[1]}", tData.index))
            tData = tData.to_frame().T
            data = pd.concat([data, tData], ignore_index=True)

         # Draw the pose annotation on the image.
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        mp_drawing.draw_landmarks(
            image,
            results.pose_landmarks,
            mp_pose.POSE_CONNECTIONS,
            landmark_drawing_spec=mp_drawing_styles.get_default_pose_landmarks_style())
        # Flip the image horizontally for a selfie-view display.
        #cv2.imshow('MediaPipe Pose', cv2.flip(image, 1))
        if exportVid:
            out.write(image)
        if 0xFF == 27:
            break

C:\Users\mmccu\venv_home_video_analysis_2\lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '
C:\Users\mmccu\venv_home_video_analysis_2\lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '
C:\Users\mmccu\venv_home_video_analysis_2\lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please 

In [15]:
if exportVid:
    out.release()

In [16]:
cap.release()

In [17]:
data.head()

,0_index,0_x,0_y,0_z,0_vis,0_pres,1_index,1_x,1_y,1_z,...,31_y,31_z,31_vis,31_pres,32_index,32_x,32_y,32_z,32_vis,32_pres
0,0.0,0.152574,0.436428,0.412269,0.988255,0.0,1.0,0.158261,0.423633,0.404737,...,0.411180,-0.805396,0.763309,0.0,32.0,0.882336,0.409877,-0.753886,0.697495,0.0
1,0.0,0.594700,0.247980,-0.197465,0.989408,0.0,1.0,0.608281,0.244941,-0.176344,...,0.923321,-0.342827,0.757601,0.0,32.0,0.365158,0.933074,-0.356524,0.675011,0.0
2,0.0,0.135898,0.405520,-0.126832,0.982912,0.0,1.0,0.118979,0.402878,-0.158411,...,0.425543,0.218341,0.702481,0.0,32.0,0.712032,0.431887,0.238734,0.621296,0.0
3,0.0,0.130616,0.407351,-0.025414,0.984375,0.0,1.0,0.113416,0.404990,-0.058421,...,0.418669,-0.166452,0.707938,0.0,32.0,0.784991,0.408618,-0.257079,0.613578,0.0
4,0.0,0.111369,0.420440,0.529604,0.984483,0.0,1.0,0.096724,0.417508,0.502486,...,0.431610,-0.518151,0.723121,0.0,32.0,0.777137,0.409061,-0.372168,0.599184,0.0


In [18]:
data.columns = [renameCols(col) for col in data.columns]
data.head()

,nose_index,nose_x,nose_y,nose_z,nose_vis,nose_pres,left_eye_inner_index,left_eye_inner_x,left_eye_inner_y,left_eye_inner_z,...,left_foot_index_y,left_foot_index_z,left_foot_index_vis,left_foot_index_pres,right_foot_index_index,right_foot_index_x,right_foot_index_y,right_foot_index_z,right_foot_index_vis,right_foot_index_pres
0,0.0,0.152574,0.436428,0.412269,0.988255,0.0,1.0,0.158261,0.423633,0.404737,...,0.411180,-0.805396,0.763309,0.0,32.0,0.882336,0.409877,-0.753886,0.697495,0.0
1,0.0,0.594700,0.247980,-0.197465,0.989408,0.0,1.0,0.608281,0.244941,-0.176344,...,0.923321,-0.342827,0.757601,0.0,32.0,0.365158,0.933074,-0.356524,0.675011,0.0
2,0.0,0.135898,0.405520,-0.126832,0.982912,0.0,1.0,0.118979,0.402878,-0.158411,...,0.425543,0.218341,0.702481,0.0,32.0,0.712032,0.431887,0.238734,0.621296,0.0
3,0.0,0.130616,0.407351,-0.025414,0.984375,0.0,1.0,0.113416,0.404990,-0.058421,...,0.418669,-0.166452,0.707938,0.0,32.0,0.784991,0.408618,-0.257079,0.613578,0.0
4,0.0,0.111369,0.420440,0.529604,0.984483,0.0,1.0,0.096724,0.417508,0.502486,...,0.431610,-0.518151,0.723121,0.0,32.0,0.777137,0.409061,-0.372168,0.599184,0.0


In [19]:
print(outFile)
print(os.path.normpath(f'./{outFile}_mpFrames.csv'))

C:\Users\mmccu\Box\MM_Personal\5_Projects\BoveLab\3_Data_and_Code\hva_code\HomeVideoAnalysis\temp\gait_vertical_left
C:\Users\mmccu\Box\MM_Personal\5_Projects\BoveLab\3_Data_and_Code\hva_code\HomeVideoAnalysis\temp\gait_vertical_left_mpFrames.csv


In [20]:
# Save the DataFrame to CSV
data.to_csv(os.path.normpath(f'./{outFile}_mpFrames.csv'))